This file is my attempt to take the code available at https://github.com/deepchem/deepchem/tree/master/contrib/mpnn and convert it to Python3 and allow it to read the data file I provided. Unfortunately, the source repo does not contain the dataset they reference and the file I found elsewhere is in a different format.

In [1]:
import math
import os

In [2]:
import deepchem as dc
from deepchem.utils import ScaffoldGenerator
from deepchem.utils.save import log
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
RDKit WARNING: [08:48:46] Enabling RDKit 2019.09.3 jupyter extensions
/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (typ

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [4]:
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
import numpy as np

In [5]:
import random
from collections import OrderedDict
from scipy.stats import pearsonr

In [18]:
import donkey_py3

ModuleNotFoundError: No module named 'donkey_py3'

In [6]:
random.seed(2)
np.random.seed(2)
torch.manual_seed(2)

In [9]:
# DATASET = 'az_ppb.csv'
DATASET = 'az_ppb_clean.csv'
print(DATASET)

az_ppb_clean.csv


In [10]:
T = 3
BATCH_SIZE = 48
MAXITER = 40000
LIMIT = 0
LR = 5e-4

In [11]:
R = nn.Linear(150, 128)
U = {0: nn.Linear(156, 75), 1: nn.Linear(156, 75), 2: nn.Linear(156, 75)}
V = {0: nn.Linear(75, 75), 1: nn.Linear(75, 75), 2: nn.Linear(75, 75)}
E = nn.Linear(6, 6)

In [12]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by .8 every 5 epochs"""
    lr = LR * (0.9 ** (epoch // 10))
    print('new lr [%.5f]' % lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [13]:
def load_dataset():
    train_features, train_labels, val_features, val_labels = donkey_py3.load_dataset(DATASET)
    
    train_labels = train_labels.reshape(-1, 1)
    val_labels = val_labels.reshape(-1, 1)
    
    scaler = preprocessing.StandardScaler().fit(train_labels)
    train_labels = scaler.transform(train_labels)
    val_labels = scaler.transform(val_labels)

    train_labels = Variable(torch.FloatTensor(train_labels), requires_grad=False)
    val_labels = Variable(torch.FloatTensor(val_labels), requires_grad=False)

    return train_features, train_labels, val_features, val_labels

In [14]:
def readout(h, h2):
    catted_reads = map(lambda x: torch.cat([h[x[0]], h2[x[1]]], 1), zip(h2.keys(), h.keys()))
    activated_reads = map(lambda x: F.selu( R(x) ), catted_reads)
    readout = Variable(torch.zeros(1, 128))
    for read in activated_reads:
        readout = readout + read
    return F.tanh(readout)

In [15]:
def message_pass(g, h, k):
    for v in g.keys():
        neighbors = g[v]
        for neighbor in neighbors:
            e_vw = neighbor[0] # feature variable
            w = neighbor[1]

            m_w = V[k](h[w])
            m_e_vw = E(e_vw)
            reshaped = torch.cat( (h[v], m_w, m_e_vw), 1)
            h[v] = F.selu(U[k](reshaped))

In [16]:
def construct_multigraph(smile):
    g = OrderedDict({})
    h = OrderedDict({})
    molecule = Chem.MolFromSmiles(smile)
    for i in range(0, molecule.GetNumAtoms()):
        atom_i = molecule.GetAtomWithIdx(i)
        h[i] = Variable(torch.FloatTensor(dc.feat.graph_features.atom_features(atom_i))).view(1, 75)
        for j in range(0, molecule.GetNumAtoms()):
            e_ij = molecule.GetBondBetweenAtoms(i, j)
            if e_ij != None:
                
                ### THIS HAS BEEN MODIFIED FROM SOURCE
#                 e_ij =  map(lambda x: 1 if x == True else 0, 
#                             dc.feat.graph_features.bond_features(e_ij)) # ADDED edge feat
                e_ij = [*map(lambda x: 1 if x == True else 0, 
                           dc.feat.graph_features.bond_features(e_ij))]
    
                e_ij = Variable(torch.FloatTensor(e_ij).view(1, 6))
                atom_j = molecule.GetAtomWithIdx(j)
                if i not in g:
                    g[i] = []
                g[i].append( (e_ij, j) )
    return g, h

In [17]:
train_smiles, train_labels, val_smiles, val_labels = load_dataset()

NameError: name 'donkey_load_dataset' is not defined

In [19]:
linear = nn.Linear(128, 1)
params = [{'params': R.parameters()},
         {'params': U[0].parameters()},
         {'params': U[1].parameters()},
         {'params': U[2].parameters()},
         {'params': E.parameters()},
         {'params': V[0].parameters()},
         {'params': V[1].parameters()},
         {'params': V[2].parameters()},
         {'params': linear.parameters()}]

In [20]:
num_epoch = 0
optimizer = optim.Adam(params, lr=LR, weight_decay=1e-4)
for i in range(0, MAXITER):
    optimizer.zero_grad()
    train_loss = Variable(torch.zeros(1, 1))
    y_hats_train = []
    for j in range(0, BATCH_SIZE):
        sample_index = random.randint(0, len(train_smiles) - 2)
        smile = train_smiles[sample_index]
        g, h = construct_multigraph(smile) # TODO: cache this

        g2, h2 = construct_multigraph(smile)

        for k in range(0, T):
            message_pass(g, h, k)

        x = readout(h, h2)
        #x = F.selu( fc(x) )
        y_hat = linear(x)
        y = train_labels[sample_index]

        y_hats_train.append(y_hat)

        error = (y_hat - y)*(y_hat - y) / Variable(torch.FloatTensor([BATCH_SIZE])).view(1, 1)
        train_loss = train_loss + error

    train_loss.backward()
    optimizer.step()

    if i % int(len(train_smiles) / BATCH_SIZE) == 0:
        val_loss = Variable(torch.zeros(1, 1), requires_grad=False)
        y_hats_val = []
        for j in range(0, len(val_smiles)):
            g, h = construct_multigraph(val_smiles[j])
            g2, h2 = construct_multigraph(val_smiles[j])

            for k in range(0, T):
                message_pass(g, h, k)

            x = readout(h, h2)
            #x = F.selu( fc(x) )
            y_hat = linear(x)
            y = val_labels[j]

            y_hats_val.append(y_hat)

            error = (y_hat - y)*(y_hat - y) / Variable(torch.FloatTensor([len(val_smiles)])).view(1, 1)
            val_loss = val_loss + error

    ## MODIFYING FROM SOURCE
#     y_hats_val = np.array(map(lambda x: x.data.numpy(), y_hats_val))
#     y_val = np.array(map(lambda x: x.data.numpy(), val_labels))
    y_hats_val = np.array(list(map(lambda x: x.data.numpy(), y_hats_val)), dtype='float64')
    y_val = np.array(list(map(lambda x: x.data.numpy(), val_labels)), dtype='float64')

    y_hats_val = y_hats_val.reshape(-1, 1)
    y_val = y_val.reshape(-1, 1)

    r2_val_old = r2_score(y_val, y_hats_val)
    r2_val_new = pearsonr(y_val, y_hats_val)[0]**2

    train_loss_ = train_loss.data.numpy()[0]
    val_loss_ = val_loss.data.numpy()[0]
    print('epoch [%i/%i] train_loss [%f] val_loss [%f] r2_val_old [%.4f], r2_val_new [%.4f]' \
                  % (num_epoch, 100, train_loss_, val_loss_, r2_val_old, r2_val_new))
    num_epoch += 1

/home/sohompaul/psi4conda/envs/deepchem/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


TypeError: No loop matching the specified signature and casting was found for ufunc add